In [18]:
import datetime

import requests
import quandl
import pandas as pd
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression
import numpy as np
import pickle

import matplotlib
import matplotlib.pyplot   as plt
from matplotlib import style
style.use('ggplot')
matplotlib.use('TkAgg')


In [19]:
df = pd.DataFrame((quandl.get('WIKI/GOOGL', authtoken='1rN-xLV-VcAMm6S64eCv')))

In [20]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]

In [21]:
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  3.712563    0.324968   44659000.0
2004-08-20   54.322689  0.710922    7.227007   22834300.0
2004-08-23   54.869377  3.729433   -1.227880   18256100.0
2004-08-24   52.597363  6.417469   -5.726357   15247300.0
2004-08-25   53.164113  1.886792    1.183658    9188600.0


In [22]:
import math

df.fillna(-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))
print(forecast_out)
df['label'] = df['Adj. Close'].shift(-forecast_out)

35


In [23]:
x = np.array(df.drop(['label'],axis=1)) 
x = preprocessing.scale(x)
x = x[:-forecast_out]
x_lately = x[-forecast_out:]
df.dropna(inplace=True) 
y = np.array(df['label'])
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2)
# clf = LinearRegression(n_jobs=-1)
# clf.fit(x_train, y_train)
# with open('linearregression1.pickle', 'wb') as handle:
#     pickle.dump(clf, handle)

pickle_in = open('linearregression1.pickle', 'rb')
clf = pickle.load(pickle_in)
accuracy = clf.score(x_test, y_test)
print(accuracy)

0.976625916434691


In [24]:
# print(x_lately)
forecast_set = clf.predict(x_lately)
# print(forecast_set, accuracy, forecast_out) 
df['Forecast'] = np.nan

last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

In [25]:
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]

print(df.tail())
df['Adj. Close'].plot()
df['Forecast'].plot()


plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

                     Adj. Close  HL_PCT  PCT_change  Adj. Volume  label  \
Date                                                                      
2018-03-08 07:00:00         NaN     NaN         NaN          NaN    NaN   
2018-03-09 07:00:00         NaN     NaN         NaN          NaN    NaN   
2018-03-10 07:00:00         NaN     NaN         NaN          NaN    NaN   
2018-03-11 07:00:00         NaN     NaN         NaN          NaN    NaN   
2018-03-12 07:00:00         NaN     NaN         NaN          NaN    NaN   

                        Forecast  
Date                              
2018-03-08 07:00:00  1199.357207  
2018-03-09 07:00:00  1204.691724  
2018-03-10 07:00:00  1203.610047  
2018-03-11 07:00:00  1138.934520  
2018-03-12 07:00:00  1086.166929  
